try to get the job data by querying 資料分析, 資料工程, 數據分析, 數據工程

In [263]:
import re, requests
from bs4 import BeautifulSoup
import csv

keyword = '數據工程' # set keyword

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
id_list = [] # lisf for saving the job_id

# due to the 100 pages limitation, we query by area
area_list = ['6001002000','6001003000','6001004000','6001005000','6001006000',
            '6001007000','6001008000','6001009000','6001010000','6001011000','6001012000',
            '6001013000','6001014000','6001015000','6001016000','6001017000','6001018000',
            '6001019000','6001020000',
            '6001001001','6001001002','6001001003','6001001004','6001001005','6001001006',
            '6001001007','6001001008','6001001009','6001001010','6001001011','6001001012',
            ]
for area in area_list:
    for page in range(0,101): #use the pages to check all the pages
        my_params = {'ro':'1', # full time
                     'keyword':'{}'.format(keyword), # set the keywords
                     'area':'{}'.format(area), # city is taipei
                     'page' : '{}'.format(page),
                     'mode':'l'} # set the listing browse mode

        response = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers)
        soup = BeautifulSoup(response.text, "lxml")
        try: # use the keyword to check if we found the last page
            re.search(r'\u5443\u62cd\u8b1d', str(soup)).group()
            print('end of the results {}'.format(area))
            break
        except:
#             print('opening the page')
            for item in soup.select('a.js-job-link'):
                full_link = item.get('href')
                link_id = re.search(r'job\/(.{5})', full_link).group(1)
                id_list.append([link_id])
print(len(id_list))

with open('job_id.csv', 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerows(id_list)

end of the results 6001002000
end of the results 6001003000
end of the results 6001004000
end of the results 6001005000
end of the results 6001006000
end of the results 6001007000
end of the results 6001008000
end of the results 6001009000
end of the results 6001010000
end of the results 6001011000
end of the results 6001012000
end of the results 6001013000
end of the results 6001014000
end of the results 6001015000
end of the results 6001016000
end of the results 6001017000
end of the results 6001018000
end of the results 6001019000
end of the results 6001020000
end of the results 6001001001
end of the results 6001001002
end of the results 6001001003
end of the results 6001001004
end of the results 6001001005
end of the results 6001001006
end of the results 6001001007
end of the results 6001001008
end of the results 6001001009
end of the results 6001001010
end of the results 6001001011
end of the results 6001001012
1508


In [209]:
import re, requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}
id_list = [] # lisf for saving the job_id
list_area = ['6001001000','6001002000','6001003000','6001004000','6001005000','6001006000',
            '6001007000','6001008000','6001009000','6001010000','6001011000','6001012000',
            '6001013000','6001014000','6001015000','6001016000','6001017000','6001018000',
            '6001019000','6001020000']

for page in range(0,100): #use the pages to check all the pages
    my_params = {'ro':'1', # full time
                 'keyword':'台灣大學', # set the keywords
                 'area':'6001001000', # city is taipei
    #              'isnew':'30', # update in last one month
                 'page' : '{}'.format(page),
                 'mode':'l'} # set the listing browse mode

    response = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers)
    soup = BeautifulSoup(response.text, "lxml")
    try: # use the keyword to check if we found the last page
        re.search(r'\u5443\u62cd\u8b1d', str(soup)).group()
        print('end of the results')
        break
    except:
        print('opening the page')
        for item in soup.select('a.js-job-link'):
            full_link = item.get('href')
            link_id = re.search(r'job\/(.{5})', full_link).group(1)
            id_list.append(link_id)
print(len(id_list))


opening the page
opening the page
opening the page
end of the results
88


In [284]:
# remove duplicate
import pandas as pd
df = pd.read_csv('job_id.csv')
df = df.drop_duplicates()
job_id_list = df.values.tolist()
print(len(job_id_list))

9710


In [ ]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'}

my_params = {'ro':'1', # 限定全職的工作，如果不限定則輸入0
             'keyword':'工程', # 想要查詢的關鍵字
             'area':'6001001000', # 限定在台北的工作
             'isnew':'30', # 只要最近一個月有更新的過的職缺
             'mode':'l'} # 清單的瀏覽模式

url = requests.get('https://www.104.com.tw/jobs/search/?' , my_params, headers = headers).url
driver = webdriver.Chrome()
driver.get(url)

for i in range(20): 
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    
k = 1
while k != 0:
    try:
        driver.find_elements_by_class_name("js-more-page").click() 
#         driver.find_element_by_xpath("//*[contains(text(),'手動載入')]").click()
        print('Click 手動載入，' + '載入第' + str(15 + k) + '頁')
        k += 1
        time.sleep(1) # 時間設定太短的話，來不及載入新資料就會跳錯誤
    except:
        k = 0
        print('No more Job')

soup = BeautifulSoup(driver.page_source, 'html.parser')
List = soup.findAll('a',{'class':'js-job-link'})
print('共有 ' + str(len(List)) + ' 筆資料')

print(len(List))

def bind(cate):
    k = []
    for i in cate:
        if len(i.text) > 0:
            k.append(i.text)
    return str(k)


# JobList = pd.DataFrame()

# i = 0
# while i < len(List):
#     print('正在處理第' + str(i) + '筆，共 ' + str(len(List)) + ' 筆資料')
#     content = List[i]
#     # 這裡用Try的原因是，有時候爬太快會遭到系統阻擋導致失敗。因此透過這個方式，當我們遇到錯誤時，會重新再爬一次資料！
# #     try:
#     print('https://' + content.attrs['href'].strip('//'))
#     resp = requests.get('https://' + content.attrs['href'].strip('//'))
#     print('準備BeautifulSoup')
#     soup2 = BeautifulSoup(resp.text,'html.parser')
#     print(soup2)
#     df = pd.DataFrame(
#         data = [{
#             '公司名稱':soup2.find('a', {'class':'cn'}).text,
#             '工作職稱':content.attrs['title'],
#             '工作內容':soup2.find('p').text,
#             '職務類別':bind(soup2.findAll('dd', {'class':'cate'})[0].findAll('span')),
#             '工作待遇':soup2.find('dd', {'class':'salary'}).text.split('\n\n',2)[0].replace(' ',''),
#             '工作性質':soup2.select('div > dl > dd')[2].text,
#             '上班地點':soup2.select('div > dl > dd')[3].text.split('\n\n',2)[0].split('\n',2)[1].replace(' ',''),
#             '管理責任':soup2.select('div > dl > dd')[4].text,
#             '出差外派':soup2.select('div > dl > dd')[5].text,
#             '上班時段':soup2.select('div > dl > dd')[6].text,
#             '休假制度':soup2.select('div > dl > dd')[7].text,
#             '可上班日':soup2.select('div > dl > dd')[8].text,
#             '需求人數':soup2.select('div > dl > dd')[9].text,
#             '接受身份':soup2.select('div.content > dl > dd')[10].text,
#             '學歷要求':soup2.select('div.content > dl > dd')[12].text,
#             '工作經歷':soup2.select('div.content > dl > dd')[11].text,
#             '語文條件':soup2.select('div.content > dl > dd')[14].text,
#             '擅長工具':soup2.select('div.content > dl > dd')[15].text,
#             '工作技能':soup2.select('div.content > dl > dd')[16].text,
#             '其他條件':soup2.select('div.content > dl > dd')[17].text,
#             '公司福利':soup2.select('div.content > p')[1].text,
#             '科系要求':soup2.select('div.content > dl > dd')[13].text,
#             '聯絡方式':soup2.select('div.content')[3].text.replace('\n',''),
#             '連結路徑':'https://' + content.attrs['href'].strip('//')}],
#         columns = ['公司名稱','工作職稱','工作內容','職務類別','工作待遇','工作性質','上班地點','管理責任','出差外派',
#                    '上班時段','休假制度','可上班日','需求人數','接受身份','學歷要求','工作經歷','語文條件','擅長工具',
#                    '工作技能','其他條件','公司福利','科系要求','聯絡方式','連結路徑'])
#     print('list加入')
#     JobList = JobList.append(df, ignore_index=True)
#     i += 1
#     print("Success and Crawl Next 目前正在爬第" + str(i) + "個職缺資訊")
#     time.sleep(1) # 執行完休息0.5秒，避免造成對方主機負擔
#     except:
#         print('break')
#         break

# JobList

# JobList.to_csv('JobList2.xlsx', encoding='cp950')


In [ ]:
import urllib.request
import urllib.parse
import gzip
import json

def ungzip(data):
    try: data=gzip.decompress(data)
    except: pass
    return data

result_list = [['jobName', 'custName', 'industry', 'workExp', 'edu', 'major',
    'language', 'localLanguage', 'specialty', 'skill', 'certificate', 'other',
    'jobDescription', 'salary', 'salaryMin', 'salaryMax', 'addressRegion',
    'addressDetail', 'manageResp', 'businessTrip', 'workPeriod', 'vacationPolicy',
    'longitude', 'latitude']]

count = 1

for job_id in job_id_list:
    count += 1
    job_id = job_id[0]
    url = 'https://www.104.com.tw/job/ajax/content/{}'.format(job_id)
    headers = {
        'Host': 'www.104.com.tw',
        'Connection': 'keep-alive',
        'Accept': 'application/json, text/plain, */*',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_1_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
        'Sec-Fetch-Site': 'same-origin',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Dest': 'empty',
        'Referer': 'https://www.104.com.tw/job/{}?jobsource=n104bank2'.format(job_id),
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7,zh-CN;q=0.6,ja;q=0.5'
        }
    try:
        req = urllib.request.Request(url,headers=headers)
        result = urllib.request.urlopen(req)
        result_str = ungzip(result.read()).decode()
        job_data = json.loads(result_str)
    
        # header
        jobName = job_data['data']['header']['jobName']
        custName = job_data['data']['header']['custName']

        # industry
        industry = job_data['data']['industry']

        # condition
        workExp = job_data['data']['condition']['workExp']
        edu = job_data['data']['condition']['edu']
        major = job_data['data']['condition']['major']
        language = job_data['data']['condition']['language']
        localLanguage = job_data['data']['condition']['localLanguage']
        specialty = []
        for i in range(0, 10):
            try: 
                specialty.append(job_data['data']['condition']['specialty'][i]['description'])
            except:
                break
        skill = job_data['data']['condition']['skill']
        certificate = job_data['data']['condition']['certificate']
        other = job_data['data']['condition']['other']

        #job detail
        jobDescription = job_data['data']['jobDetail']['jobDescription']
        salary = job_data['data']['jobDetail']['salary']
        salaryMin = job_data['data']['jobDetail']['salaryMin']
        salaryMax = job_data['data']['jobDetail']['salaryMax']
        addressRegion = job_data['data']['jobDetail']['addressRegion']
        addressDetail = job_data['data']['jobDetail']['addressDetail']
        manageResp = job_data['data']['jobDetail']['manageResp']
        businessTrip = job_data['data']['jobDetail']['businessTrip']
        workPeriod = job_data['data']['jobDetail']['workPeriod']
        vacationPolicy = job_data['data']['jobDetail']['vacationPolicy']
        longitude = job_data['data']['jobDetail']['longitude']
        latitude = job_data['data']['jobDetail']['latitude']

        job_row = [jobName, custName, industry, workExp, edu, major, language, localLanguage,
        specialty, skill, certificate, other, jobDescription, salary, salaryMin,
        salaryMax, addressRegion, addressDetail, manageResp, businessTrip,
        workPeriod, vacationPolicy, longitude, latitude]

        result_list.append(job_row)
        print(count)
    except:
        print(url)
        continue
    
with open('job_rawdata.csv', 'a', newline = '') as f:
    writer = csv.writer(f)
    writer.writerows(result_list)

In [293]:
job_df = pd.read_csv('job_rawdata.csv')
job_df.describe()

,salaryMin,salaryMax,longitude,latitude
count,9.699000e+03,9.699000e+03,9698.000000,9698.000000
mean,3.036044e+04,8.861038e+05,121.051329,24.597408
std,1.033590e+05,2.777543e+06,4.286330,1.135837
min,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,120.717015,24.246434
50%,2.800000e+04,3.500000e+04,121.485683,25.024944
75%,3.300000e+04,5.000000e+04,121.549783,25.054159
max,3.000000e+06,9.999999e+06,121.853326,25.243263
